## Analisi di 22.5 Milioni di valutazioni di libri su Amazon
In questo notebook utilizzeremo un RDD di Spark per analizzare circa 22.5 milioni di valutazioni di libri prese da Amazon.com.
<br>
Le domande alla quale risponderemo sono le seguenti.

* Quante valutazioni ci sono nel dataset ?
* Quanti libri ci sono nel dataset ?
* Quante valutazioni ha ricevuto ogni libro ?
* Quali sono i 10 libri più valutati ?
* Qual è la valutazione media per ogni libro ?
* Quali sono i 10 libri con la valutazione più alta ?
* Chi sono i 10 recensori più critici ?

## Procuriamoci il Dataset
Per prima cosa procuriamoci il dataset scaricandolo il locale, il dataset si trova in formato CSV a [questo link](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Books.csv), esegui il comando qui sotto per scaricarlo direttamente da Jupyter.

In [1]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Books.csv

--2019-07-04 10:05:54--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Books.csv
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 916259348 (874M) [text/csv]
Saving to: ‘ratings_Books.csv.1’

ratings_Books.csv.1 100%[===================>] 873.81M  3.52MB/s    in 4m 8s   

2019-07-04 10:10:02 (3.52 MB/s) - ‘ratings_Books.csv.1’ saved [916259348/916259348]



## Inizializziamo Spark

In [20]:
from pyspark import SparkConf, SparkContext

In [6]:
conf = SparkConf().setMaster("local").setAppName("AmazonReviews")
sc = SparkContext(conf=conf)

## Importiamo il CSV dentro un RDD
Per caricare un qualsiasi file di testo dentro un RDD possiamo usare il metodo *.textFile(path)* della nostra istanza della classe SparkContext.

In [21]:
reviewsRDD = sc.textFile("ratings_Books.csv")

Vediamo un po' cosa contiene l'RDD. Avendo 22.5 milioni di elementi, utilizzare il metodo *.collect()* è sconveniente per ovvie ragioni, al suo posto possiamo usare il metodo *.take(n)* per selezionare soltanto n elementi.

In [22]:
reviewsRDD.take(5)

['AH2L9G3DQHHAJ,0000000116,4.0,1019865600',
 'A2IIIDRK3PRRZY,0000000116,1.0,1395619200',
 'A1TADCM7YWPQ8M,0000000868,4.0,1031702400',
 'AWGH7V0BDOJKB,0000013714,4.0,1383177600',
 'A3UTQPQPM4TQO0,0000013714,5.0,1374883200']

Ogni elemento di ogni riga corrisponde a (in ordine):
* Id dell'utente che ha lasciato la valutazione.
* Id del libro recensito.
* Valutazione da 1.0 a 5.0.
* Timestamp di quando è stata lasciata la recensione.

Per ovvi motivi di privacy non ci è possibile risalire ad un utente partendo dal suo ID, mentre per il libro è possibile farlo aggiungendo l'ID a questo url https://www.amazon.com/dp/, ad esempio per il primo elemento: https://www.amazon.com/dp/0000000116
<br><br>
**NOTA BENE** Sì lo so, se hai cliccato sul link ti sei trovato una penna e non un libro come abbiamo detto, il motivo è che tale penna è stata inserita impropriamente nella categoria libri di Amazon, quindi tutto in regola per noi :).

## Contiamo il numero totale di valutazioni
Per contare il numero di recensioni usiamo semplicemente il metodo *.count()*

In [23]:
reviewsRDD.count()

22507155

Quindi in realtà sono più di 2.5 milioni di libri nel dataset, buono a sapersi :D.

## Contiamo le valutazioni per ogni libro
Per contare le valutazioni che ogni libro ha ricevuto creiamo un nuovo RDD contenente soltanto gli ID dei libri per ogni riga. 

In [24]:
productsRDD = reviewsRDD.map(lambda x: x.split(",")[1])
productsRDD.take(5)

['0000000116', '0000000116', '0000000868', '0000013714', '0000013714']

Poi usiamo semplicemente il metodo *.countByValue()*

In [25]:
productsCount = productsRDD.countByValue()

Stampiamo il numero di valutazioni ricevute per i primi 10 libri.

In [26]:
i = 0
print("ID LIBRO\tCONTEGGIO")
for product_id, count in productsCount.items():
    print("%s\t%s" % (product_id, count))
    if(i>=10):
        break
    i+=1

ID LIBRO	CONTEGGIO
0000000116	2
0000000868	1
0000013714	14
0000015393	1
0000029831	5
0000038504	2
0000041696	4
0000095699	1
0000174076	1
0000202010	1
0000230022	10


## Troviamo i 10 libri più valutati
Per trovare i 10 libri più valutati potremmo semplicemente utilizzare il defaultdict ottenuto sopra, però voglio farti vedere un'altro modo per farlo !
<br>
Mappiamo ogni elemento ad una lista, contenente l'elemento stesso ed un valore 1 (lo so sembra una cosa senza senso, ma è un'operazione tipica del processo map-reduce, ora capirai perché).

In [27]:
productsCount = productsRDD.map(lambda x: (x, 1))
productsCount.take(5)

[('0000000116', 1),
 ('0000000116', 1),
 ('0000000868', 1),
 ('0000013714', 1),
 ('0000013714', 1)]

Utilizziamo il metodo *reduceByKey* per sommare i valori degli elementi aventi la stessa chiave.

In [28]:
productsCount = productsCount.reduceByKey(lambda x, y: x+y)
productsCount.take(5)

[('0001006657', 2),
 ('0001922408', 2),
 ('0002000601', 6),
 ('0002006650', 2),
 ('0002007770', 6001)]

Capito il trucchetto ? Riducendo l'RDD tramite una somma dei valori 1 che abbiamo aggiunto prima abbiamo ottenuto la somma totale delle valutazioni per ogni libro. Ora ci basta ordinarli in senso decrescente e tenere stampare i primi 10 risultati.

In [29]:
productsCountSorted = productsCount.sortBy(lambda x: x[1], ascending=False)
productsCountSorted.take(10)

[('0439023483', 21398),
 ('030758836X', 19867),
 ('0439023513', 14114),
 ('0385537859', 12973),
 ('0007444117', 12629),
 ('0375831002', 12571),
 ('038536315X', 12564),
 ('0345803485', 12290),
 ('0316055433', 11746),
 ('0849922070', 10424)]

Ecco qui i 10 libri più recensiti, qui possiamo vedere i primi 3:
* https://www.amazon.com/dp/0439023483
* https://www.amazon.com/dp/030758836X
* https://www.amazon.com/dp/0439023513

Il primo è The Hunger Games, leggilo se non lo hai già fatto (o al massimo guarda il film).
<br><br>
**NOTA BENE**
<br>
Se il numero delle valutazione che vedi nel sito dovesse essere superiore rispetto a quello riportato dalla nostra analisi, non preoccuparti, non abbiamo fatto nulla di sbagliato, semplicemente gli utenti di Amazon hanno lasciato nuove valutazioni rispetto a quando il dataset che stiamo usando è stato creato.

## Calcoliamo la valutazione media
Per calcolare la valutazione media creiamo un nuovo RDD contenete soltanto ID del libro e valutazione.

In [30]:
def parseProductRating(row):
    columns = row.split(",")
    product = columns[1]
    rating = float(columns[2])
    
    return (product, rating)

productsRDD = reviewsRDD.map(parseProductRating)
productsRDD.take(5)

[('0000000116', 4.0),
 ('0000000116', 1.0),
 ('0000000868', 4.0),
 ('0000013714', 4.0),
 ('0000013714', 5.0)]

Proviamo a somamre il totale delle valutazioni usando il metodo reduceByKey.

In [31]:
ratingSumRDD = productsRDD.reduceByKey(lambda x,y: x+y)
ratingSumRDD.take(5)

[('0001006657', 10.0),
 ('0001922408', 10.0),
 ('0002000601', 23.0),
 ('0002006650', 8.0),
 ('0002007770', 26398.0)]

Ora dovremmo dividere per il numero di valutazioni che ogni libro ha ricevuto, ma eseguendo la riduzione abbiamo perso questa informazione, quindi non è la cosa giusta da fare.
<br><br>
Proviamo in un'altro modo, mappiamo ogni elemento ad una lista, contenente l'elemento stesso ed un valore 1 che ci servirà come contatore, esattamente come fatto in precedenza, poi eseguiamo la riduzione per chiave sommando sia i contatori che le valutazione come fatto appena sopra. Per non farci mancare niente (e per non mettere troppi RDD in memoria) facciamo tutto in un unica istruzione.

In [32]:
ratingSumRDD = productsRDD.mapValues(lambda x: (x,1)).reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
ratingSumRDD.take(5)

[('0001006657', (10.0, 2)),
 ('0001922408', (10.0, 2)),
 ('0002000601', (23.0, 6)),
 ('0002006650', (8.0, 2)),
 ('0002007770', (26398.0, 6001))]

Perfetto ! Ora abbiamo sia la somma che il conteggio, quindi possiamo eseguire un map di nuovo, dividendo il secondo per il primo.

In [33]:
ratingMeanRDD = ratingSumRDD.mapValues(lambda x: x[0]/x[1])
ratingMeanRDD.take(5)

[('0001006657', 5.0),
 ('0001922408', 5.0),
 ('0002000601', 3.8333333333333335),
 ('0002006650', 4.0),
 ('0002007770', 4.398933511081486)]

## Troviamo i 10 libri con la valutazione più alta
Per trovare i libri con la valutazione più alta potremmo semplicemente ordinare l'RDD calcolato appena sopra, però otterremo dei risulati falsati, dato che libri che hanno ottenuto un'unica valutazione a 5 stelle saranno alle prime posizioni. Quindi facciamo una cosa più intelligente considerando solo i libri che sono stati valutati almeno 100 volte.<br><br>
Calcoliamo nuovamente la valutazione media per ogni libro, questa volta però teniamo anche il conteggio.

In [34]:
ratingMeanRDD = ratingSumRDD.mapValues(lambda x: (x[0]/x[1], x[1]))
ratingMeanRDD.take(5)

[('0001006657', (5.0, 2)),
 ('0001922408', (5.0, 2)),
 ('0002000601', (3.8333333333333335, 6)),
 ('0002006650', (4.0, 2)),
 ('0002007770', (4.398933511081486, 6001))]

Filtriamo i libri, tenendo soltanto quelli che hanno avuto almeno 100 valutazioni, per curiosità contiamo quanti sono.

In [35]:
ratingMeanRDD = ratingMeanRDD.filter(lambda x: x[1][1]>=100)
ratingMeanRDD.count()

29296

Quasi 3mila, non pochi ! Ora ordiniamo quest'ultimo RDD in base alla valutazione media e stampiamo i primi 10 risultati.

In [36]:
ratingSortedRDD = ratingMeanRDD.sortBy(lambda x: x[1][0], ascending=False)
ratingSortedRDD.take(10)

[('0983408904', (5.0, 128)),
 ('0830766316', (5.0, 103)),
 ('0972394648', (4.992647058823529, 136)),
 ('1499390165', (4.991803278688525, 122)),
 ('0849381185', (4.990566037735849, 106)),
 ('0757317723', (4.9862068965517246, 145)),
 ('1939629071', (4.983193277310924, 119)),
 ('1499381921', (4.982857142857143, 350)),
 ('1616387165', (4.981308411214953, 107)),
 ('0814416993', (4.980769230769231, 104))]

Ed ecco qui le valutazioni medie per ogni libro !

Ecco qui i 10 libri più recensiti, qui possiamo vedere i primi 3:
* https://www.amazon.com/dp/0983408904
* https://www.amazon.com/dp/0830766316
* https://www.amazon.com/dp/0972394648
<br><br>

**NOTA BENE**
<br>
Ovviamente sarebbe stato più intelligente eseguire la riduzione tenendo anche il conteggio la prima volta, quando abbiamo calcolato la valutazione media per libro per rispondere alla domanda precedente, in modo tale da non dover rieseguire un'operazione abbastanza dispendiosa, ma dato che il nostro scopo è imparare abbiamo fatto meglio a far così :).

## Troviamo i 10 recensori più critici
Cerchiamo i 10 recensori più critici, cioè quelli che sono soliti lasciare le recensioni più basse, per farlo calcoliamo la valutazione media lasciata da ogni recensore e ordiniamo l'RDD così ottenuto in maniera ascendente. Il processo è quasi identico a quello che abbiamo già eseguito per ottenere i libri con la valutazione maggiore.
<br><br>
Comciamo creando un nuovo dataset contenente soltanto id utente come chiave e valutazione e 1 come valore, l'1 ci serve sempre come contatore.

In [37]:
def parseReviewerRating(row):
    columns = row.split(",")
    reviewer = columns[0]
    rating = float(columns[2])
    
    return (reviewer, (rating, 1))

reviewerRDD = reviewsRDD.map(parseReviewerRating)
reviewerRDD.take(5)

[('AH2L9G3DQHHAJ', (4.0, 1)),
 ('A2IIIDRK3PRRZY', (1.0, 1)),
 ('A1TADCM7YWPQ8M', (4.0, 1)),
 ('AWGH7V0BDOJKB', (4.0, 1)),
 ('A3UTQPQPM4TQO0', (5.0, 1))]

E sommiamo tutte le valutazioni e il contatore.

In [38]:
reviewerRDD = reviewerRDD.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
reviewerRDD.take(5)

[('A2742OG8PK8KU6', (10.0, 2)),
 ('A2GKR2Q7MD8DG4', (12.0, 3)),
 ('A1MC4E00RO5E9T', (17.0, 4)),
 ('A3IKTM9D8RVWKU', (5.0, 1)),
 ('A3UZSIDE90JWW1', (5.0, 1))]

Anche in questo caso, per evitare di ottenere soltanto i recensori che hanno lasciato un'unica valutazione, filtriamo il dataframe tenendo soltanto i recensori che hanno lasciato almeno 100 valutazioni.

In [39]:
reviewerRDD = reviewerRDD.filter(lambda x: x[1][1]>100)
reviewerRDD.count()

11244

Ne abbiamo oltre 11mila, vediamo tra questi chi sono i più cattivi, calcoliamo la loro valutazione media.

In [42]:
criticalReviewerRDD = reviewerRDD.mapValues(lambda x: x[0]/x[1])
criticalReviewerRDD.take(5)

[('A8IPQ1Q1O7YX5', 4.227048371174728),
 ('A2PN65B6BSTIYZ', 3.953271028037383),
 ('AX724J32HPG1J', 4.184738955823293),
 ('AFFGYGNO989PD', 4.2785714285714285),
 ('A1WCJEZS66D224', 3.5789473684210527)]

Ordiniamo il dataset così ottenuto in maniera ascendente e stampiamo i primi 100.

In [43]:
criticalReviewerSortedRDD = criticalReviewerRDD.sortBy(lambda x: x[1])
criticalReviewerSortedRDD.take(10)

[('AH62BQTCMR3BR', 1.0534188034188035),
 ('A186OSXC7LHJDB', 1.2014925373134329),
 ('A2HESNQJZ9OB7H', 1.2543859649122806),
 ('A36IQRD3B5MK8G', 1.505050505050505),
 ('A3JF63XRSLLH0P', 1.5648148148148149),
 ('A344N0X5LIV43M', 1.646551724137931),
 ('A1SS16UHYW77D4', 1.855421686746988),
 ('A19UFCMSFGOZ2K', 2.076923076923077),
 ('A1NJHOGKZZRAX8', 2.1588785046728973),
 ('A1ZY08GYVIKZFM', 2.2446043165467624)]

AH62BQTCMR3BR sei una brutta perzona !